In [1]:
import StockPriceHandler as sp
import StockNewsScrapper as sn
import numpy as np
import pandas as pd
from datetime import date
from datetime import datetime

2022-08-13 15:32:39,445 loading file C:\Users\amshuk\.flair\models\sentiment-en-mix-distillbert_4.pt


In [2]:
# Get Ticker Tape News Scores
df_TtNewsScores = sn.GetTickerTapeNewsSentiment('Training')

C:\Personal\Google Drive\Next Level\ISB\Term 2\FP1\FP1-SVP\StockNewsScrapper.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrameToAnalyze["Sentiment_Score"][eachItemIndex] = scoreValueFromMinus1ToPlus1


In [3]:
# Get Investing.com News Scores
df_InvNeScores = pd.read_csv("SentimentAvg_Investing_News.csv")
df_InvNeScores['Date'] = pd.to_datetime(df_InvNeScores['Date'])

In [4]:
# Merge News Scores from TickerTape & Investing.com Dataset
df_ConsNewsScore = pd.merge(left = df_TtNewsScores, right = df_InvNeScores, how = 'outer', on = ['Stock_Code', 'Date'])
# Take average of News Scores to have one single News Sentiment Score
conditions = [df_ConsNewsScore.Sentiment_Score_mean.notnull() & df_ConsNewsScore.Mean_Sentiment_Score.notnull(), df_ConsNewsScore.Sentiment_Score_mean.isnull(), df_ConsNewsScore.Mean_Sentiment_Score.isnull()]
choices = [(df_ConsNewsScore.Mean_Sentiment_Score + df_ConsNewsScore.Sentiment_Score_mean)/2, df_ConsNewsScore.Mean_Sentiment_Score, df_ConsNewsScore.Sentiment_Score_mean]
df_ConsNewsScore['News_Score'] = np.select(conditions, choices)
# Drop Individual News Scores
df_ConsNewsScore = df_ConsNewsScore.drop(['Mean_Sentiment_Score', 'Sentiment_Score_mean'], axis=1)

In [5]:
# Get Twitter Sentiment Scores
df_TwitSent = pd.read_csv("TwitterSentimentsAlldates.csv")
df_TwitSent['Date'] = pd.to_datetime(df_TwitSent['Date'])
df_TwitSent = df_TwitSent.groupby(['stock_code', 'Date']).agg({'Sentiment_Score': ['mean']})
df_TwitSent = df_TwitSent.reset_index()
df_TwitSent.columns = df_TwitSent.columns.to_flat_index().str.join('_')
df_TwitSent.rename(columns = {'stock_code_' : 'Stock_Code' , 'Date_': 'Date'}, inplace = True)

In [6]:
# Merge Twitter Sentiment Scores with News Score
df_NewsNTwitScore = pd.merge(left = df_TwitSent, right = df_ConsNewsScore, how = 'outer',left_on=['Stock_Code', 'Date'], right_on = ['Stock_Code', 'Date'])

In [7]:
list_outdatedRows = df_NewsNTwitScore[df_NewsNTwitScore['Date'] < '2022-7-26'].index
df_NewsNTwitScore.drop(list_outdatedRows, inplace = True)
df_NewsNTwitScore.reset_index()

,index,Stock_Code,Date,Sentiment_Score_mean,News_Score
0,0,ADANIPORTS,2022-07-26,0.080000,-0.564883
1,1,ADANIPORTS,2022-07-27,0.081382,NaN
2,2,ADANIPORTS,2022-07-28,0.036365,NaN
3,3,ADANIPORTS,2022-07-29,0.026564,NaN
4,4,ADANIPORTS,2022-07-30,0.029235,NaN
...,...,...,...,...,...
853,1013,WIPRO,2022-08-12,NaN,0.004709
854,1120,BAJAJFINSV,2022-07-26,NaN,0.370767
855,1197,CIPLA,2022-07-26,NaN,0.000000
856,1351,HDFCBANK,2022-07-26,NaN,0.350667


In [8]:
df_NewsNTwitScore['Sentiment_Score_mean'] = df_NewsNTwitScore['Sentiment_Score_mean'].fillna(0)
df_NewsNTwitScore['News_Score'] = df_NewsNTwitScore['News_Score'].fillna(0)

In [9]:
# Get Stock Price Data
df_StkPr = sp.GetStockPriceMetrics('Training', date(2022,7,27), date(2022,8,10), '')
df_StkPr['Date'] = pd.to_datetime(df_StkPr['Date'])

In [10]:
# Set the Outcome variable based on the value of the Price Movement
# To have a fairly balanced data for all 3 types of outcomes, threshold is set as -0.25% and +0.25%
conditions = [df_StkPr.Prediction < 0.00000001, df_StkPr.Prediction > 0.00000001]
choices = [-1, 1]
df_StkPr['Outcome'] = np.select(conditions, choices)
# Drop the ununsed columns
df_StkPr = df_StkPr.drop(['Open','Close','MA5', 'Prediction'], axis = 1)

In [11]:
# Merge Stock Price with Twitter and News Scores
df_TrainingSet = pd.merge(left = df_StkPr, right = df_NewsNTwitScore, how = 'inner',left_on=['Stock_Code', 'Date'], right_on = ['Stock_Code', 'Date'])

In [12]:
df_TrainingSet.to_csv("TrainingSet_Binary.csv")